# Array Broadcasting: What is x+y?


In [1]:
import numpy as np

x = np.array([[0], [1], [2]])
y = np.array([[3, 4, 5]])

In [2]:
print(x)
print(y)

[[0]
 [1]
 [2]]
[[3 4 5]]


In [3]:
x + y

array([[3, 4, 5],
       [4, 5, 6],
       [5, 6, 7]])

# Array Broadcasting

A sensible way of doing elementwise operations on arrays of different (but compatible) shapes.

$$\pmatrix{0 & 1  & 2\\ 3 & 4 & 5} + \pmatrix{1 & 2 & 3} = \pmatrix{1 & 3  & 5\\ 4 & 6 & 8}$$

$$\pmatrix{0 & 1  & 2\\ 3 & 4 & 5} + \pmatrix{1 \\ 2} = \pmatrix{1 & 2 & 3 \\ 5 & 6 & 7}$$

It works with plus, minus, times, exponentiation, min/max, and many more elementwise operations. Search the numpy docs for the word "broadcast" to see if it is supported.

# Shape Compatibility Rules

1. If x, y have a different number of dimensions, prepend 1's to the shape of the shorter.
2. Any axis of length 1 can be repeated (broadcast) to the length of the other vector's length in that axis
3. All other axes must have matching lengths.

Use these rules to compute whether the arrays are compatible and, if so, the broadcasted shape.

# Example 1

In [ ]:
x.shape == (2, 3)

y.shape == (2, 3)  # compatible
y.shape == (2, 1)  # compatible
y.shape == (1, 3)  # compatible
y.shape == (3,)  # compatible

# results in (2, 3) shape

y.shape == (3, 2)  # NOT compatible
y.shape == (2,)  # NOT compatible

# Example 2

In [ ]:
x.shape == (1000, 256, 256, 256)

y.shape == (1000, 256, 256, 256)  # compatible
y.shape == (1000, 1, 256, 256)  # compatible
y.shape == (1000, 1, 1, 256)  # compatible
y.shape == (1, 256, 256, 256)  # compatible
y.shape == (1, 1, 256, 1)  # compatible

# results in (1000, 256, 256, 256) shape

y.shape == (1000, 256, 256)  # NOT compatible

# Example 3

In [ ]:
x.shape == (1, 2, 3, 5, 1, 11, 1, 17)
y.shape ==          (1, 7, 1,  1, 17)  # compatible

# results in shape (1, 2, 3, 5, 7, 11, 1, 17)

# Once shapes match, use for-loop to understand

For any axis with length 1, use the only possible value.

In [4]:
x = np.array([[0, 1, 2],
              [3, 4, 5],
              [6, 7, 8]])
y = np.array([1, 10, 100]).reshape(3, 1)

print(x + y)

# x     (3, 3)
# y     (3, 1)
shape = (3, 3)
out = np.empty(shape, dtype=int)
N0, N1 = shape
for i in range(N0):
    for j in range(N1):
        # in the dimension that y only has 1 element, just use it
        out[i, j] = x[i, j] + y[i, 0]
print(out)

[[  1   2   3]
 [ 13  14  15]
 [106 107 108]]
[[  1   2   3]
 [ 13  14  15]
 [106 107 108]]


Just omit variables for prepended 1's.

In [6]:
x = np.array([[[0, 1, 2],
               [3, 4, 5],
               [6, 7, 8]],
              [[9, 10, 11],
               [12, 13, 14],
               [15, 16, 17]]])  # shape (2, 3, 3)
y = np.array([1, 10, 100])     # shape       (3,)

print(x + y)

# align and prepend
# x     (2, 3, 3)
# y     (1, 1, 3)
shape = (2, 3, 3)
out = np.empty(shape, dtype=int)
N0, N1, N2 = shape
for i in range(N0):
    for j in range(N1):
        for k in range(N2):
            # leave off prepended indices of y
            out[i, j, k] = x[i, j, k] + y[k]
print(out)

[[[  1  11 102]
  [  4  14 105]
  [  7  17 108]]

 [[ 10  20 111]
  [ 13  23 114]
  [ 16  26 117]]]
[[[  1  11 102]
  [  4  14 105]
  [  7  17 108]]

 [[ 10  20 111]
  [ 13  23 114]
  [ 16  26 117]]]


Both arrays can have broadcasted axes, not just one.

In [7]:
x = np.array([[0], [1], [2]])  # (3, 1)
y = np.array([[3, 4, 5]])     # (1, 3)

print(x + y)

shape = (3, 3)
out = np.empty(shape, dtype=int)
N0, N1 = shape
for i in range(N0):
    for j in range(N1):
        out[i, j] = x[i, 0] + y[0, j]
print(out)

[[3 4 5]
 [4 5 6]
 [5 6 7]]
[[3 4 5]
 [4 5 6]
 [5 6 7]]


# Exercise for you

You have N=1000 images of size WxH=32x32, where each image has C=3 channels (red, green, and blue pixel value from 0 to 255) for each location in the image.

Suppose you have the images stored in an array x of size (N, C, W, H) == (1000, 3, 32, 32)

What array y would you multiply by to scale every red pixel by 2, every green pixel by 3, and every blue pixel by 4 (don't worry about overflow)?

In [ ]:
# Answer
y = np.array([2, 3, 4]).reshape(1, 3, 1, 1)
# or
# y = np.array([2, 3, 4]).reshape(3, 1, 1)

# Understand with for-loops
x = np.ones(shape, dtype=np.uint8) # for example
shape = (1000, 3, 32, 32)
out = np.empty(shape, dtype=np.uint8)
N, C, W, H = shape
for n in range(N):
    for channel in range(C):
        for w in range(W):
            for h in range(H):
                out[n, channel, w, h] = x[n, channel, w, h] * y[0, channel, 0, 0]